<h4>A: Data Dashboards</h4>
<h5>Datasets<h5>
<p>The datasets will both be provided with the submission, but they contain
<ul>
<li>WGU-provided telecommunications dataset</li>
<li><a href="https://www.kaggle.com/datasets/aadityabansalcodes/telecommunications-industry-customer-churn-dataset?select=Telco_customer_churn_status.csv">Kaggle-provided telecommunications dataset</a></li>
</ul>
</p>
<h5>Installation instructions</h5>
<p>The dashboard is being provided in a .twbx format, and as such, will need to be opened in Tableau Desktop. The steps to access the dashboard will be as follows:
<ol>
<li>Download the Tableau file uploaded with the submission</li>
<li>Sign into Tableau public at this link: <a href="https://www.tableau.com/products/public/download"> Tableau download link</a></li>
<li>Select role and download the installer</li>
<li>Once the installer finishes downloading, open the installer</li>
<li>Accept the terms and conditions, then click install at the bottom. A popup may appear, asking for permission. If it does, click accept</li>
<li>A progress bar will now show up. Once it completes, Tableau public will be installed on the computer. Use this application to open the .twbx file included with this submission
</ol>
</p>
<h5>Navigation instructions</h5>
<p>Navigating the dashboard should be simple, as all of the information is contained within a single story. At the top, there are different tabs one can click on to navigate between the pages, as well as arrows on either side to go through the dashboard page-by-page. Each page, aside from the landing page, contains 2 graphics and 1 filter each. The graphics are labeled, to denote which one is showing external and WGU data, making them easy to distinguish. The filters come in 3 types: a checkbox (on Tenure by Geographic Region), in which the user can click on the boxes to filter based on churn status, a slider (on churn and customer count by bandwidth), allowing the user to determine a maximum and minimum value for bandwidth usage, and a dropdown menu (on Avg Tenure By Contract Duration) which allows the user to filter down the payment type by clicking to expand the menu, then selecting the different options</p>
<h5>Copy of SQL Code</h5>
<p>A complete copy of the SQL code can be found below. This code will bring the data from the csv files into the PostgreSQL Database, join the public data to each other, join the WGU data tables together, and then create a union between the two datasets, with a column denoting the data source:</p>


<p>-- Create Table: external_data_churn
CREATE TABLE public.external_data_churn (
    status_id TEXT,
    customer_id TEXT PRIMARY KEY,
    count INTEGER,
    quarter TEXT,
    satisfaction_score INTEGER,
    customer_status TEXT,
    churn_label TEXT,
    churn_value INTEGER,
    churn_score INTEGER,
    CLTV INTEGER,
    churn_category TEXT,
    churn_reason TEXT
);
-- Create Table: external_data_demographics
CREATE TABLE public.external_data_demographics (
    customer_id TEXT,
    count INTEGER,
    gender TEXT,
    age INTEGER,
    under_30 TEXT,
    senior_citizen TEXT,
    married TEXT,
    dependents TEXT,
    number_of_dependents INTEGER,
	CONSTRAINT fk_customer FOREIGN KEY (customer_id) REFERENCES public.external_data_churn (customer_id) ON DELETE CASCADE
);



-- Create Table: external_data_location
CREATE TABLE public.external_data_location (
    location_id TEXT PRIMARY KEY,
    customer_id TEXT NOT NULL,
    count INTEGER,
    country TEXT,
    state TEXT,
    city TEXT,
    zip_code TEXT,
    lat_long TEXT,
    latitude NUMERIC,
    longitude NUMERIC,
    CONSTRAINT fk_customer FOREIGN KEY (customer_id) REFERENCES public.external_data_churn (customer_id) ON DELETE CASCADE
);

-- Create Table: external_data_services
CREATE TABLE public.external_data_services (
    service_id TEXT PRIMARY KEY,
    customer_id TEXT NOT NULL,
    count INTEGER,
    quarter TEXT,
    referred_a_friend TEXT,
    number_of_referrals INTEGER,
    tenure_in_months INTEGER,
    offer TEXT,
    phone_service TEXT,
    avg_monthly_long_distance_charges NUMERIC,
    multiple_lines TEXT,
    internet_service TEXT,
    internet_type TEXT,
    avg_monthly_gb_download NUMERIC,
    online_security TEXT,
    online_backup TEXT,
    device_protection_plan TEXT,
    premium_tech_support TEXT,
    streaming_tv TEXT,
    streaming_movies TEXT,
    streaming_music TEXT,
    unlimited_data TEXT,
    contract TEXT,
    paperless_billing TEXT,
    payment_method TEXT,
    monthly_charge NUMERIC,
    total_charges NUMERIC,
    total_refunds NUMERIC,
    total_extra_data_charges NUMERIC,
    total_long_distance_charges NUMERIC,
    total_revenue NUMERIC,
    CONSTRAINT fk_customer FOREIGN KEY (customer_id) REFERENCES public.external_data_churn (customer_id) ON DELETE CASCADE
);

-- Set table ownership to postgres
ALTER TABLE public.external_data_churn OWNER TO postgres;
ALTER TABLE public.external_data_services OWNER TO postgres;
ALTER TABLE public.external_data_demographics OWNER TO postgres;
ALTER TABLE public.external_data_location OWNER TO postgres;

-- Load Data
COPY external_data_churn
FROM 'C:\Users\Public\Downloads\Telco_customer_churn_status.csv'
DELIMITER ','
CSV HEADER;

COPY external_data_services
FROM 'C:\Users\Public\Downloads\Telco_customer_churn_services.csv'
DELIMITER ','
CSV HEADER;

COPY external_data_demographics
FROM 'C:\Users\Public\Downloads\Telco_customer_churn_demographics.csv'
DELIMITER ','
CSV HEADER;

COPY external_data_location
FROM 'C:\Users\Public\Downloads\Telco_customer_churn_location.csv'
DELIMITER ','
CSV HEADER;

-- Query with CTEs
CREATE TABLE Combined_Dataset AS(
WITH WGU_data AS (
    SELECT customer.customer_id::TEXT, tenure::INTEGER, bandwidth_gp_year::NUMERIC, duration::TEXT, 
        payment_type::TEXT, monthly_charge::NUMERIC, age::INTEGER, city::TEXT, state::TEXT, 
        zip::TEXT, lat::NUMERIC, lng::NUMERIC, churn::TEXT, 'Provided Data' AS source 
    FROM customer
    LEFT JOIN payment ON payment.payment_id = customer.payment_id
    LEFT JOIN contract ON customer.contract_id = contract.contract_id
    LEFT JOIN location ON customer.location_id = location.location_id
    LEFT JOIN job ON job.job_id = customer.job_id
),
External_data AS (
    SELECT external_data_churn.customer_id::TEXT, tenure_in_months::INTEGER AS tenure, 
        (avg_monthly_gb_download * 12)::NUMERIC AS bandwidth_gp_year, contract::TEXT AS duration, 
        payment_method::TEXT AS payment_type, total_revenue::NUMERIC AS monthly_charge, 
        age::INTEGER, city::TEXT, state::TEXT, zip_code::TEXT AS zip, 
        latitude::NUMERIC AS lat, longitude::NUMERIC AS lng, 
        customer_status::TEXT AS churn, 'External Data' AS source 
    FROM external_data_churn
    LEFT JOIN external_data_services ON external_data_churn.customer_id = external_data_services.customer_id
    LEFT JOIN external_data_demographics ON external_data_demographics.customer_id = external_data_churn.customer_id
    LEFT JOIN external_data_location ON external_data_location.customer_id = external_data_churn.customer_id
)
SELECT * FROM WGU_data
UNION ALL 
SELECT * FROM External_data);
</p>



<h4>B: Demonstration</h4>
<h5>Link</h5>
<p>The Panopto video displaying the dashboard and usage is available at the following link: <a href="https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=b14da18d-b9c8-40eb-8d88-b28201206409">Panopto Presentation</a></p>
<h4>C: Report</h4>
<h5>Purpose and Function</h5>
<p>The primary purpose of the dashboard is to provide insight into customer churn and tenure, including the factors that might contribute to churn rate, both within the company in the WGU dataset and in the wider industrial environment. This is useful to the stakeholders primarily due to the high cost of customer acquisition in the telecommunications industry. Identifying the factors likely to result in longer customer tenure and lower churn rates allows the business to attempt to maximize the amount of time that the customer stays with the business, thus improving business profitability.</p>
<h5>Justifying Business Tool</h5>
<p>The tool that I decided to use for this project is Tableau. This is useful for a number of reasons. First, the ability to make a story in Tableau enables those with minimal technological skills to access and utilize the dashboard without issue, as the user only needs to click on the relevant page to see all of the information. Furthermore, Tableau has great mechanisms to deal with colorblindness and other accessibility issues, such as having high-contrast colors by default and allowing users to hover on areas to get a black-and-white textual breakdown. Finally, Tableau has an option to publish information online, which could be useful for the business going forward, as it would allow users to access it without needing to worry about software installation or file versions.</p>
<h5>Steps to clean and prepare the data</h5>
<p>The data cleaning and preparation took place over the course of several steps. First, the new data needed to be ingested into the PostgreSQL database. During this step, we will create primary and foreign key constraints and ensure these columns do not contain null values, using the NOT NULL keyword. Once these tables are created and populated, the next step is to combine the tables from each dataset together (so all the WGU tables are joined, and all of the external tables are joined), which will create a complete table for each data source. This will be done using the keys that we had already made, ensuring referential integrity. Finally, we will create a union between the WGU and external data, allowing both of these datasets to be accessed in one combined table, with the data source being marked in its own column. This process also involves including only those columns that are similar between the two tables, as well as casting the data types and aliasing the columns to match. Once this is complete, we have one table capable of being used in the Tableau Dashboard.</p>
<h5>Steps to create the dashboard</h5>
<p>There are a few steps that were required to create the dashboard. First, the data needed to be ingested into the Tableau Desktop application. Next, I made a calculated field to reduce the options for the churn column to only contain yes and no values, as this would be useful in creating a binary way of viewing the data going forward. Similarly, we would want to make the Bandwidth_gp_year column into a bin column, rather than continuous. This will help us to group customers together, with a bin size of 150. Next, there are a few pages to create. For each story point, we need to create 1 dashboard page, which requires the creation of 2 worksheets. The process is to create a worksheet for the external data for each visualization (Geographic Tenure, Churn and Customer Count by Bandwidth, and Avg Tenue By Contract Duration). Each of these pages will have a filter  built in to restrict the data source, and a usable filter added for the relevant data on each page. Each page, after being created, can be duplicated, then have the filter switched to the WGU datasource to view that data instead. The map of tenure is created by using state as the columns entry, and avg tenure for the rows. Then, clicking on show me will allow the user to select a map to display the data. The bandwidth column chart is created using the buckets for bandwidth previously mentioned as the columns, and the count of the binary churn column used as the rows. The churn is also used for color marks, so we can see how many users did and did not churn within each bucket. Tenure by contract is then created in a similar manner, but using duration and average tenure instead. The matching graphics are added to a single dashboard page, stacked one over the other, and these dashboard pages are then copied onto the relevant story point.</p>
<h5>Results</h5>
<p>We have a few results we can discuss with the findings. First, the company in the WGU dataset performs significantly better at customer retention than the Kaggle dataset, indicating that the WGU company is performing incredibly well compared to the wider industry. This matches with the purpose of the dashboard, as one of our goals was to understand how the business performs with regards to customer retention. Furthermore, we can see that customers with high bandwidth usage are far less likely to churn. This comports with the function of the dashboard, as it identifies one of the key metrics that determines customer retention. The business can use this information to identify possible customers with high data usage and create plans that incentivize high data usage, helping to retain customers over the long term.</p>
<h5>Limitations</h5>
<p>The primary limitation of the analysis is the lack of variables available. The dataset provided by WGU does not contain many variables that are useful in determining service options that might entice and retain customers. Having more information regarding different service options would allow a more interactive dashboard that can consider more possible reasons for customer retention and recruitment. Another limitation is that the external dataset only contains data for California. I would rather the external dataset be nationwide, to provide a comparison of how all states perform in each, to see if the trends are the same nationwide in both datasets, or whether there is some sort of anomaly or discrepancy.</p>

<h4>Web Sources</h4>
<p><a href="https://neon.tech/postgresql/postgresql-tutorial/import-csv-file-into-posgresql-table#import-csv-file-into-a-table-using-pgadmin">Neon Tech Tutorial: Import CSV File Into PostgreSQL Table</a></p>
<h4>Referenced Content Sources</h4>
<p><a href="https://app.datacamp.com/learn/custom-tracks/custom-d211-advanced-data-acquisition">DataCamp Track used to understand the usage of joins and keys</a></p>
